In [ ]:
# Core 
import datetime
import os
import glob
from IPython.display import HTML, Image, display
import tempfile
import shutil

# Analysis 
import xarray as xr
import numpy as np
import pyproj as pp
import scipy as sp
import metpy as mp
import networkx 
from networkx.algorithms.components.connected import connected_components

# Plotting 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
import matplotlib.colors as colors

# Debugging 
import pdb
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif

In [ ]:
# Store filenames for an example day of data
filenames = sorted(glob.glob('/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161220/*.nc'))
# Generate grid generator (generators produce iterators - alternative to using lists and looping)
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)

# Define settings for tracking
settings = {
    'MIN_SIZE' : 30, 
    'FIELD_THRESH' : 20,
    'GS_ALT' : 1500,
    'LEVELS' : np.array(
        [[0, 3000], [3000, 6000]]
    )
}

# Calculate high and low level tracks
tracks_obj  = Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS']:
    tracks_obj.params[parameter] = settings[parameter]

# Calculate tracks
tracks_obj.get_tracks(grids)

In [ ]:
upper = np.array([
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 3, 3, 0, 0, 0, 4, 4],
    [0, 3, 3, 0, 0, 0, 4, 4],
])

lower = np.array([
    [0, 1, 1, 1, 1, 1, 1, 4],
    [0, 1, 1, 1, 1, 1, 1, 4],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 3, 3, 3, 0, 2, 2, 2],
    [0, 3, 3, 3, 0, 2, 2, 2],
    [0, 3, 3, 3, 0, 2, 2, 2],
])

In [ ]:
print(upper.max())

overlapped_objects = [set(range(1, lower.max()+1))] * upper.max()
print(overlapped_objects)

for i in range(upper.max()):
    overlapped_objects[i] = set(
        (upper.max() + lower[np.logical_and(upper == (i+1), lower > 0)]).flatten()
    )
    print(str(i))
    print(set(lower[upper==(i+1)].flatten()))
    
overlapped_objects
# Loop through overlapped objects.
    # 

    

In [ ]:
range(1,upper.max()+1)

In [ ]:
list(overlapped_objects[i])[0]

In [ ]:
len(overlapped_objects[1])

In [ ]:
overlap_graph = networkx.Graph()
overlap_graph.add_nodes_from(set(range(1, upper.max()+1)))
for i in range(0, len(overlapped_objects)):
    overlap_graph.add_nodes_from(overlapped_objects[i])
    overlap_graph.add_edges_from([(i+1,list(overlapped_objects[i])[0])])
    for j in range(0, len(overlapped_objects[i])-1):
        overlap_graph.add_edges_from(
            [(list(overlapped_objects[i])[j], list(overlapped_objects[i])[j+1])]
        )
list(connected_components(overlap_graph))

In [ ]:
overlap_graph.edges

In [ ]:
set(range(1, upper.max()+1))

In [ ]:
len(overlapped_objects[2].intersection(overlapped_objects[1]))>0

In [ ]:
tracks_obj.tracks

In [ ]:
# Specify rounded bounds of CPOL 1000 m gridded dataset 
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)

tmp_dir = tempfile.mkdtemp()

# Animate low level tracks
animate(tracks_obj, grids, os.path.join(tmp_dir, 'tracks_anim'),
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True)

embed_mp4_as_gif(os.path.join(tmp_dir, 'tracks_anim.mp4'))

shutil.rmtree(tmp_dir)

In [ ]:
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:10]

In [ ]:
# Lagrangian Animation
tmp_dir = tempfile.mkdtemp()

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator
animate(tracks_obj, grids, os.path.join(tmp_dir, 'lagrangian'), style='lagrangian', uid='151', alt=1500)

embed_mp4_as_gif(os.path.join(tmp_dir, 'lagrangian.mp4'))
shutil.rmtree(tmp_dir)

In [ ]:
test[0,1]

In [ ]:
test=np.array([[10, 20], [2, 30]])